# OpenAI 适配器

对于部分已经适配 OpenAI 而尚未支持千帆 SDK 的第三库，本文提供了一种方法，可以快速将千帆适配至任意这类库。

千帆 SDK CLI 支持启动一个将千帆模拟成 OpenAI 接口的服务，然后让 OpenAI 请求该服务即可完成千帆的接入。

具体来说，首先需要在命令行中运行 `qianfan openai` 启动一个模拟服务。

可以通过 `-p 8001` 来指定端口，以及 `-d` 来让服务保持在后台运行。

当服务启动后，只需要设置两个环境变量，就可以在代码中直接调用支持 OpenAI 的第三方库，这里以 langchain 为例：

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "any-content"  # 任意内容，仅为绕过 OpenAI SDK 验证
os.environ["OPENAI_BASE_URL"] = "http://127.0.0.1:8001/v1"  # 修改为本地 OpenAI API 地址

In [3]:
from langchain_openai import OpenAI
from langchain_core.messages import HumanMessage, SystemMessage

llm = OpenAI()

llm.invoke([
    HumanMessage(
        content="你是谁"
    ),
])

'你好，我是百度研发的文心一言。我可以回答你的问题或与你对话。请问你有什么具体的问题或需要帮助吗？'

也支持使用 function_call 实现 Agent

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

In [3]:
tools = [DuckDuckGoSearchRun()]
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [4]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "文心千帆是什么"})



> Entering new AgentExecutor chain...
文心千帆是百度推出的一款企业级知识增强大模型，旨在帮助企业实现智能化决策和精细化运营。它基于百度强大的飞桨深度学习平台和文心知识增强大模型，能够为企业提供丰富的语义理解和知识推理能力，帮助企业从海量数据中挖掘出有价值的信息，提升企业的智能化水平。

文心千帆提供了多种应用场景，如智能客服、智能推荐、智能问答、智能文本分类等，能够为企业提供全方位的智能化服务。同时，文心千帆还支持定制化开发，企业可以根据自身需求，定制符合自身业务场景的模型，实现更加精细化的运营。

总之，文心千帆是一款功能强大、易于使用、灵活定制的企业级知识增强大模型，能够帮助企业实现智能化决策和精细化运营，提升企业的竞争力和创新能力。

> Finished chain.


{'input': '文心千帆是什么',
 'output': '文心千帆是百度推出的一款企业级知识增强大模型，旨在帮助企业实现智能化决策和精细化运营。它基于百度强大的飞桨深度学习平台和文心知识增强大模型，能够为企业提供丰富的语义理解和知识推理能力，帮助企业从海量数据中挖掘出有价值的信息，提升企业的智能化水平。\n\n文心千帆提供了多种应用场景，如智能客服、智能推荐、智能问答、智能文本分类等，能够为企业提供全方位的智能化服务。同时，文心千帆还支持定制化开发，企业可以根据自身需求，定制符合自身业务场景的模型，实现更加精细化的运营。\n\n总之，文心千帆是一款功能强大、易于使用、灵活定制的企业级知识增强大模型，能够帮助企业实现智能化决策和精细化运营，提升企业的竞争力和创新能力。'}